# Generative AI: Customize Chatbot with new LLM Models
NeuralChat is a customizable chat framework designed to create user own chatbot within few minutes on multiple architectures. This notebook will introduce how to use your new LLM model to customize chatbot.

## Register the new Model 💻

Create `new_llm_model.py` in ***intel_extension_for_transformers/neural_chat/models/***, and define `NewModel` class derived from `BaseModel` in `base_model.py`.

In [ ]:
from .base_model import BaseModel, register_model_adapter
import logging
from fastchat.conversation import Conversation

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

class NewModel(BaseModel):
    def match(self, model_path: str):
        """
        Check if the provided model_path matches the current model.

        Args:
            model_path (str): Path to a model.

        Returns:
            bool: True if the model_path matches, False otherwise.
        """
        return "new_model" in model_path.lower()

    def get_default_conv_template(self, model_path: str) -> Conversation:
        """
        Get the default conversation template for the given model path.

        Args:
            model_path (str): Path to the model.

        Returns:
            Conversation: A default conversation template.
        """
        new_conv_template = Conversation(name="new_model")
        return new_conv_template

register_model_adapter(NewModel)

## Create Model Adapter 💡

Create model adapter in ***intel_extension_for_transformers/neural_chat/chatbot.py*** at line 51. Modify the identification token like `new_llm_model`.

In [ ]:
if "llama" in config.model_name_or_path.lower():
    from .models.llama_model import LlamaModel
    adapter = LlamaModel()
# create new model adapter here
elif "new_llm_model" in config.model_name_or_path:
    from .models.new_llm_model import NewModel
    adapter = NewModel()
else:
    raise ValueError("NeuralChat Error: Unsupported model name or path, \
                        only supports FLAN-T5/LLAMA/MPT/GPT/BLOOM/OPT/NEURAL-CHAT now.")

## Customize Chatbot 🚀

Customize chatbot with the new llm model. Change the `model_name_or_path` to the real path and make sure the token "new_llm_model" is in it.

In [ ]:
from intel_extension_for_transformers.neural_chat import build_chatbot
from intel_extension_for_transformers.neural_chat.config import PipelineConfig
config = PipelineConfig(model_name_or_path="/path/to/new_llm_model")
chatbot = build_chatbot(config)
response = chatbot.predict(query="Tell me about Intel Xeon Scalable Processors.")
print(response)